In [1]:
# Importing modules needed
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
import re
import random
import pandas as pd

[nltk_data] Downloading package punkt to /Users/yososky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df_neg = pd.read_csv("negativeTweets.csv")
df_neg.head()  

,negativereason,airline,tweet_location,retweet_count,text,sentiment,topic
0,Bad Flight,Virgin America,NaN,0,@VirginAmerica it's really aggressive to blast...,Negative,3
1,Can't Tell,Virgin America,NaN,0,@VirginAmerica and it's a really big bad thing...,Negative,8
2,Can't Tell,Virgin America,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,Negative,8
3,NaN,Virgin America,San Diego,0,"@virginamerica Well, I didn'tâ€¦but NOW I DO! :-D",Negative,7
4,NaN,Virgin America,1/1 loner squad,0,@VirginAmerica did you know that suicide is th...,Negative,6


## Function for fitting the problem to the right reason

First Step:
import all the libraries required to process
then import the data-set “Tweets.csv” 
 
Second step:
Train the model on the bases of text ,negativereason and tweet_location

third step:
find out the occurrence of location from tweet_location and we see that most of the time people don’t mention there location and the reset top5 location [(\'unknown\', 293) , (\'San Francisco\', 27),(\'San Francisco, CA\', 21),(\'USA\', 15),(\'Seattle\', 15)]  that are more frequently use are noted to be use during process. (Optional)

Fourth step:
Apply Decision Tree Classifier to get the results of the tweet that user will enter
it will determine the negativereason and tweet lcation

In [3]:
import pandas as pd, numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re, string
df_train = pd.read_csv("Tweets.csv")
df_train["negativereason"].fillna("unknown", inplace=True)
df_train["tweet_location"].fillna("unknown", inplace=True)
df_train["text"].fillna("unknown", inplace=True)
text_train = df_train.iloc[: , 6].values
labels_train = df_train.iloc[: , [1,4]].values
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
vect = TfidfVectorizer(use_idf = False ,tokenizer=tokenize,smooth_idf = True )
matrix_x = vect.fit_transform(text_train)
DT = DecisionTreeClassifier()
weight = DT.fit(matrix_x, labels_train)

In [9]:
## use for 
text_input = input("Enter the Text: ")
text = []
text.append(text_input)
pre_x = vect.transform(text)
labels = weight.predict(pre_x)
print(labels[0][0])

Enter the Text: by booked flight is getting error message
Can't Tell


## Auto read system
We have set up a auto read system as well. Since the data itself has told us enought infomation, we can skip this section. But if we are facing new sentances that not included in the system. The fuction will be appled

In [5]:
# Created mutiple categories for Eliza's response
# Regluar Expression is also used to search for user's input- 
"""
Cancelled Flightled (cancelled flightled)
Bad Flight:  (seat)
Late Flight: (late, delay)
booking issue (book, error message)
Lost/Damaged Luggage (lost baggage, luggage, damaged)
Flight Attendant Complaints (attendant)
customer service: (service, email, website)
"""
eliza_dict= { 
    
    r'.*(cancelled flightled|cancelled flight|cancel|Cancelled Flight)(.*)':
        [
            "If you meet problem with your flight schedule, please find us at our counter for your schedule!",
            "We are sorry to hear your issue. But! You can call use with xxx. We will help you with your flight!"
         ],   
    
    r'(.*)(minutes late flight|late|delay|delayed|Bad Flight).*':
        [
            "We understand our servcie may bring hurt to your time. Visit www.usgift.com and get your free drink with cupon XXX today!",
            "Still waiting for you flight? We prepare free snack for you! Visit www.usgift.com with cupon XXX!",
            "If you are boring for waiting, come to our counter and pick your free nuts!"
         ],  
    
    r'(.*) (seat|seats|wifi|Bad Flight) (.*)':
        [
            "We understand your are facing seating issue. The carefully and fairly treat every customer. If you feel opposite, please call our number xxx",
            "Sometimes, its hard for us to fit everyone into the wished seat. But! Tips! Your can double check when boarding!",
            "Maybe you missed your best seat. A good tip is to check seat before boarding. Maybe we can upgrade for you!"
        ],
    
    r'(.*) (book|rebook|booked|booking|error message|Flight Booking Problems)(.*)':
        [
            "Did you meet problem with booking? Contact our customer support for further help! Number is xxxxxx.",
            "If you meet booking issue, best way is call use or email us with your situation. We will reply you soon!",
            "Have you receive the comfrim email recipt? If not, please contact us now!"
        ],
    
    r'(.*) (lost my bag|baggage|luggage|damaged|Lost Luggage)(.*)':
        [
            "If you meet baggage issue, you can find our baggage center for help!",
            "Are you facing baggae issue? Call number xxxxx-xxx. Our baggage center will help you with your issue!"
        ],
    
    r'(.*) (attendant|Flight Attendant Complaints) (.*)':
        [
            "If you meet problem with our staff? Can you tell us more detail? Call xxxx!",
            "Do you think our sevice have any improment? Please tell us at our counter!"
         ],   
    
    r'(.*)(service|customer service|email|website|Customer Service Issue)(.*)':
        [
            "Our customer manager is open to you if you really need our help. You can call xxx dail with xxx directly!",
            "Can you tell us your email or phone number here (www.xxxxxx.com) with issue xxx. We will have someone help you soon."
         ],
    
    r'(.*)':
         [
            "You can visit our website xxx for more help! We have online represenatitives!",
            "Please tell us more with us to help use improve. Call xxx today!",        
            "Meet our staff at the counter. Tell us your story. We will help you!"
        ]
}

In [27]:
#Main function to check user's input and allows auto reply with proper response
def figureAirline (userTyped):
    tokens = word_tokenize(userTyped)
    if tokens[0] == '@':
        airline = tokens[1]
    else:
        airline = 'Aviation Alliance'
    return airline

def userSideInput(userTyped, with_airline):  
    # find the paired re from the dictionary
    for matches, responses in eliza_dict.items():   
        ans = re.search(matches, userTyped.rstrip(",.").lower())   
        
        # if found a match (userinput = key value in Eliza_dict)
        if ans:       
            
            # choose a random reply                                  
            reply = random.choice(responses)      
            if "%s" in reply:
                sWord = ans[1]
                #print(sWord)
                pronouce = pronounceSwitch(sWord)
                reply = reply %(pronouce)
            if with_airline:
                return '[Message from ' + str(figureAirline(userTyped)) + ']: ' + str(reply)
            else:
                return str(reply)

In [29]:
for i in range(3):
    message = df_neg.iloc[random.randint(0, len(df_neg)-1)][4]
    print('Message: ' + message)
  
    print('Reply: ' + userSideInput(message, True))
    print('\n')

Message: @united doesn't help the 100 or more passengers that just arrived an hour Late Flight.
Reply: [Message from united]: Still waiting for you flight? We prepare free snack for you! Visit www.usgift.com with cupon XXX!


Message: @USAirways has the worst customer service line.  I've called them 8 times today and not once was I able to talk to a real human being.
Reply: [Message from USAirways]: Our customer manager is open to you if you really need our help. You can call xxx dail with xxx directly!


Message: @united this is nuts they said a flight was on the ground now it's still in Albany and they won't have a plane.  Someone is compensating me
Reply: [Message from united]: You can visit our website xxx for more help! We have online represenatitives!




In [32]:
## use for text out of database. User can type complaint directly

text_input = input("Enter the Text: ")
text = []
text.append(text_input)
pre_x = vect.transform(text)
labels = weight.predict(pre_x)
print('Type of problem: ' + str(labels[0][0]))
print(userSideInput(labels[0][0], False))

Enter the Text: my fight is delayed
Type of problem: Late Flight
Still waiting for you flight? We prepare free snack for you! Visit www.usgift.com with cupon XXX!
